In [2]:
import os
import json
import pandas as pd
import numpy as np
from pprint import pprint
from pandas.core.common import flatten

변수 스케일링 방법(!!)

참고: https://sebastianraschka.com/faq/docs/scale-training-test.html
1. 학습데이터와 실험데이터 구분
2. 학습데이터의 특정 변수 스케일링 (fit)
3. 학습데이터의 특정 변수에 스케일링 적용(transform)
4. (스케일링된 학습데이터로 모델 학습) - 모델은 스케일링된 데이터를 통해 학습됨 
5. 실험데이터의 특정 변수(학습데이터에서 스케일링 적용한 변수와 동일) 스케일링 적용(transform, fit하지 않음)
6. (스케일링된 실험데이터로 모델 적용)



In [45]:
current_dir=os.getcwd()
data_dir = os.path.join(os.path.join(current_dir, 'example'),'bike_demand')
data_dir
bike = pd.read_csv(data_dir+'\\bike_train.csv')
bike.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [41]:
from sklearn.preprocessing import StandardScaler
sd_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))

from sklearn.preprocessing import MaxAbsScaler
ma_scaler = MaxAbsScaler(copy=True)

from sklearn.preprocessing import RobustScaler
rb_scaler = RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True, with_scaling=True)


In [42]:
def get_transformer(params):
    module_ = __import__(params['LIBRARY_NAME'])
    class_ = getattr(module_, params['LIBRARY_OBJECT_NAME'])
    transformer_ = getattr(class_, params['LIBRARY_FUNCTION_NAME'])
    transformer  = transformer_()
    return transformer   

In [61]:
pfunc_01 = dict(pk=1, LIBRARY_NAME='sklearn', LIBRARY_OBJECT_NAME='preprocessing', LIBRARY_FUNCTION_NAME='StandardScaler')
pfunc_02 = dict(pk=2, LIBRARY_NAME='sklearn', LIBRARY_OBJECT_NAME='preprocessing', LIBRARY_FUNCTION_NAME='MinMaxScaler')
pfunc_03 = dict(pk=3, LIBRARY_NAME='sklearn', LIBRARY_OBJECT_NAME='preprocessing', LIBRARY_FUNCTION_NAME='OneHotEncoder')
pfunc = {}
pfunc['1'] = pfunc_01
pfunc['2'] = pfunc_02
pfunc['3'] = pfunc_03
pfunc

{'1': {'LIBRARY_FUNCTION_NAME': 'StandardScaler',
  'LIBRARY_NAME': 'sklearn',
  'LIBRARY_OBJECT_NAME': 'preprocessing',
  'pk': 1},
 '2': {'LIBRARY_FUNCTION_NAME': 'MinMaxScaler',
  'LIBRARY_NAME': 'sklearn',
  'LIBRARY_OBJECT_NAME': 'preprocessing',
  'pk': 2},
 '3': {'LIBRARY_FUNCTION_NAME': 'OneHotEncoder',
  'LIBRARY_NAME': 'sklearn',
  'LIBRARY_OBJECT_NAME': 'preprocessing',
  'pk': 3}}

In [12]:
COLUMN = input('변수선택:')
TRANSFROMER = input('변환기선택(처리방식):')
print('선택된 변수 [{}] 를 [{}] 를 이용해서 변환합니다.'.format(COLUMN, TRANSFROMER))

변수선택:atemp
변환기선택(처리방식):minmax
선택된 변수 [atemp] 를 [minmax]를 이용해서 변환합니다.


In [46]:
DATA = bike.copy()
COLUMN = 'atemp'

In [22]:
unseen_data = DATA[COLUMN][:5].values.reshape(-1,1)
unseen_data

array([[14.395],
       [13.635],
       [13.635],
       [14.395],
       [14.395]])

In [11]:
DATA.head(2) 

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [5]:
request = {}
request['req_01'] = dict(pfunc_pk=1, field_name='atemp', condition='')
request['req_02'] = dict(pfunc_pk=2, field_name='humidity', condition={'feature_range':(-1,1)})
pprint(request)

{'req_01': {'condition': '', 'field_name': 'atemp', 'pfunc_pk': 1},
 'req_02': {'condition': {'feature_range': (-1, 1)},
            'field_name': 'humidity',
            'pfunc_pk': 2}}


In [389]:
DF = pd.DataFrame()
NUM = 5 
for user_request in request:
    #print(request[user_request]) #{'pfunc_pk': 1, 'field_name': 'atemp', 'condition': ''}
    get_request_dict = request[user_request]
    get_pk = get_request_dict['pfunc_pk']
    get_pfunc = pfunc[str(get_pk)]
    #print(get_pfunc) #{'pk': 1, 'LIBRARY_NAME': 'sklearn', 'LIBRARY_OBJECT_NAME': 'preprocessing', 'LIBRARY_FUNCTION_NAME': 'StandardScaler'}
    transformer = get_transformer(get_pfunc)
    get_field = DATA[get_request_dict['field_name']][:NUM]
    #print('get   default  transformer=> ', transformer)
    print('get  user  request   field=> ', get_request_dict['field_name'])
    get_condition = get_request_dict['condition']
    transformer = change_params(transformer, get_condition)
    print('set user request condition=> ', transformer)
    get_field=get_field.astype(float)
    try:
        transformer.fit(get_field)
        changed_f1 = transformer.fit_transform(get_field)
    except:
        transformer.fit(get_field.values.reshape(-1,1))
        changed_f1 = transformer.fit_transform(get_field.values.reshape(-1,1))
    #print(list(flatten(changed_f1)))#NUM])
    #changed_data[get_request_dict['field_name']]= list(flatten(changed_f1))  #:NUM]
    result = changed_f1
    to_df = pd.DataFrame(result, columns=[get_request_dict['field_name']])
    DF = pd.concat([DF, to_df], axis=1)
    print('==============done==============\n')
    
DF

get  user  request   field=>  atemp
set user request condition=>  StandardScaler(copy=True, with_mean=True, with_std=True)
==============done==============

get  user  request   field=>  humidity
set user request condition=>  MinMaxScaler(copy=True, feature_range=(-1, 1))
==============done==============

get  user  request   field=>  temp


KeyError: 'condition'

In [318]:
DF.to_dict(orient='records')

[{'atemp': 0.816496580927727, 'humidity': 1.0},
 {'atemp': -1.224744871391588, 'humidity': 0.6666666666666643},
 {'atemp': -1.224744871391588, 'humidity': 0.6666666666666643},
 {'atemp': 0.816496580927727, 'humidity': -1.0},
 {'atemp': 0.816496580927727, 'humidity': -1.0}]

In [58]:
request = {}
request['req_01'] = dict(pfunc_pk=1, field_name='atemp', condition='')
request['req_02'] = dict(pfunc_pk=2, field_name='humidity', condition={'feature_range':(-1,1)})
request['req_03'] = dict(pfunc_pk=1, field_name='temp')
request['req_04'] = dict(pfunc_pk=3, field_name='weather', condition={'categories':'auto', 'sparse':False})

## string to dictionary (08.30)

In [49]:
full_req = '{"PREPROCESS_FUNCTIONS_SEQUENCE_PK":1, "field_name":"temp", "condition":"feature_range:(-1,1)"}' #, sth:30
full_dict = json.loads(full_req)
con = full_dict['condition']
con = con.lower()
con 

'feature_range:(-1,1)'

In [50]:
def inspect_condition(params_str):
    condition_list = params_str.split(', ')
    params_dict = {}
    for request_conidtion in condition_list:
        key_, values_ = request_conidtion.split(':')[0], request_conidtion.split(':')[1]
        params_dict[key_] = values_
    return params_dict

get_params_dict = inspect_condition(con)
get_params_dict

{'feature_range': '(-1,1)'}

In [51]:
from ast import literal_eval
from distutils import util # to handle str to bool conversion


tf = MinMaxScaler()
print('오리지널', tf.get_params())
x = {}
for param_key_, param_values_ in get_params_dict.items():
    if param_values_ == 'true' or param_values_ == 'false':
        param_values_ = bool(util.strtobool(param_values_))
        #param_values_ = (False, True)[param_values_ == "True"]
    elif param_values_.startswith('('):
        param_values_ = literal_eval(param_values_)
    x[param_key_] = param_values_

tf = tf.set_params(**x)
print('파라미터 변경 후', tf.get_params())
tf.fit_transform(DATA[['atemp', 'temp']])

오리지널 {'copy': True, 'feature_range': (0, 1)}
파라미터 변경 후 {'copy': True, 'feature_range': (-1, 1)}


array([[-0.38986464, -0.55102041],
       [-0.42387292, -0.59183673],
       [-0.42387292, -0.59183673],
       ...,
       [-0.32207182, -0.34693878],
       [-0.254279  , -0.34693878],
       [-0.28828728, -0.3877551 ]])

In [52]:
r_list = [-0.5510204081632654, -0.5918367346938775, -0.5918367346938775, -0.5510204081632654, -0.5510204081632654]
r_list

[-0.5510204081632654,
 -0.5918367346938775,
 -0.5918367346938775,
 -0.5510204081632654,
 -0.5510204081632654]

In [57]:
list(map(lambda x: format(x, '.4f'), r_list))

['-0.5510', '-0.5918', '-0.5918', '-0.5510', '-0.5510']

In [56]:
list(map(lambda x: x*2, [1,2,3,4]))

[2, 4, 6, 8]

In [71]:
full_req = '{"PREPROCESS_FUNCTIONS_SEQUENCE_PK":1, "field_name":"temp", "condition":"copy:False, feature_range:(0,1), sth:30"}'
#, "condition":"{"copy":"False"}"'
print(full_req)
full_req = full_req.encode("utf-8")
print(full_req)
full_dict = json.loads(full_req)
type(full_dict)
print("full_dict['condition']",full_dict['condition'])
con = full_dict['condition']
print(type(con))


tf = MinMaxScaler()
print(tf)
tf_scaled = change_params(tf, con)
print(tf_scaled)

{"PREPROCESS_FUNCTIONS_SEQUENCE_PK":1, "field_name":"temp", "condition":"copy:False, feature_range:(0,1), sth:30"}
b'{"PREPROCESS_FUNCTIONS_SEQUENCE_PK":1, "field_name":"temp", "condition":"copy:False, feature_range:(0,1), sth:30"}'
full_dict['condition'] copy:False, feature_range:(0,1), sth:30
<class 'str'>
MinMaxScaler(copy=True, feature_range=(0, 1))
MinMaxScaler(copy=True, feature_range=(0, 1))


In [39]:
tf = MinMaxScaler()
print(tf)
full_dict = dict(copy=False)
print(full_dict)
tf_scaled = change_params(tf, full_dict)
print(tf_scaled)

MinMaxScaler(copy=True, feature_range=(0, 1))
{'copy': False}
MinMaxScaler(copy=False, feature_range=(0, 1))


In [731]:
full_req = '{"PREPROCESS_FUNCTIONS_SEQUENCE_PK":1, "field_name":"temp", "condition":"{"copy":False}"}' #"{feature_range:(-1, 1)}"
full_dict = json.loads(full_req)
print(type(full_dict["condition"]))
print(full_dict['condition'])
con = full_dict["condition"]
print(con)
n, v = con.strip('{}').split(":")
con_dict = {}
con_dict[n] = v
print('con_dict', con_dict)
#a_dict = dict([con.strip('{}').split(":"),])
#print(a_dict)
#a_dict['feature_range']

tf = MinMaxScaler()
print(tf)
for k, v in param.items():
    setattr(tf, k, v)
tf.fit_transform(DATA[['atemp', 'temp']])


JSONDecodeError: Expecting ',' delimiter: line 1 column 76 (char 75)

In [13]:
con = '{"feature_range": "(-1, 1)", "copy":"False"}'
con = json.loads(con)
tf = MinMaxScaler()
params = {}
print(tf)
for name, values in con.items():
    params[name] = values
    
tf

MinMaxScaler(copy=True, feature_range=(0, 1))


MinMaxScaler(copy=True, feature_range=(0, 1))

In [679]:
model = MinMaxScaler()
for k, v in param.items():
    setattr(model, k, v)
model.fit_transform(DATA[['atemp', 'temp']])

array([[-0.38986464, -0.55102041],
       [-0.42387292, -0.59183673],
       [-0.42387292, -0.59183673],
       ...,
       [-0.32207182, -0.34693878],
       [-0.254279  , -0.34693878],
       [-0.28828728, -0.3877551 ]])

In [ ]:
model.

In [665]:
type(param['feature_range'])

tuple

In [672]:
param = dict(feature_range=(-1,1))
print(param)
model = MinMaxScaler()

for k, v in param.items():
    setattr(model, k, v)
model

{'feature_range': (-1, 1)}


MinMaxScaler(copy=True, feature_range=(-1, 1))

In [639]:
print(MinMaxScaler())
cclf = change_params(MinMaxScaler(), con)
cclf.fit_transform(DATA[['atemp', 'temp']])

MinMaxScaler(copy=True, feature_range=(0, 1))


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [59]:
pprint(request) 

{'req_01': {'condition': '', 'field_name': 'atemp', 'pfunc_pk': 1},
 'req_02': {'condition': {'feature_range': (-1, 1)},
            'field_name': 'humidity',
            'pfunc_pk': 2},
 'req_03': {'field_name': 'temp', 'pfunc_pk': 1},
 'req_04': {'condition': {'categories': 'auto', 'sparse': False},
            'field_name': 'weather',
            'pfunc_pk': 3}}


In [62]:
pprint(pfunc) 

{'1': {'LIBRARY_FUNCTION_NAME': 'StandardScaler',
       'LIBRARY_NAME': 'sklearn',
       'LIBRARY_OBJECT_NAME': 'preprocessing',
       'pk': 1},
 '2': {'LIBRARY_FUNCTION_NAME': 'MinMaxScaler',
       'LIBRARY_NAME': 'sklearn',
       'LIBRARY_OBJECT_NAME': 'preprocessing',
       'pk': 2},
 '3': {'LIBRARY_FUNCTION_NAME': 'OneHotEncoder',
       'LIBRARY_NAME': 'sklearn',
       'LIBRARY_OBJECT_NAME': 'preprocessing',
       'pk': 3}}


In [63]:
MinMaxScaler()

MinMaxScaler(copy=True, feature_range=(0, 1))

In [153]:
oh = OneHotEncoder()
oh_trf = oh.fit_transform(DATA['weather'].values.reshape(-1,1)).toarray()
oh_trf[5631:]


c:\users\daumsoft\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [159]:
oh = OneHotEncoder() 
mm = MinMaxScaler()
#oh_trf = oh.fit_transform(DATA['temp'].values.reshape(-1,1)) 

In [160]:
def test_transformer(field, transformer):
    NUM = 5
    try:
        transformer.fit(field) #필드가 두 개 이상일 때 
        changed_field = transformer.transform(field)
    except:
        transformer.fit(field.values.reshape(-1,1)) #필드가 하나일 때 
        changed_field = transformer.transform(field.values.reshape(-1,1))

    print(changed_field[:NUM].shape)
    return changed_field[:NUM]


r_ = test_transformer(DATA['weather'], oh)
print(type(r_))
if r_.shape == (5,1): #numerical 전처리의 경우에 해당 
    changed_field = list(map(lambda x: "%.4f" %x, list(flatten(r_[:5]))))
else: #그 외 (예, 원핫인코딩)에 해당 
    #r_ = r_.toarray()
    r_ = r_.toarray() if not isinstance(r_, np.ndarray) else r_ #sp.sparse.csr_matrix
    print(type(r_))
    changed_field = list(map(lambda x:str(x), r_[:5]))
    
changed_field

(5, 4)
<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>


['[1. 0. 0. 0.]',
 '[1. 0. 0. 0.]',
 '[1. 0. 0. 0.]',
 '[1. 0. 0. 0.]',
 '[1. 0. 0. 0.]']

In [120]:
import scipy as sp

In [123]:
print(isinstance(re_, np.ndarray), isinstance(r_, sp.sparse.csr_matrix))

True True


In [135]:
ss = r_.A
ss

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [116]:
re_ = np.array([1,2,3])

#print('one') if isinstance(re_, np.ndarray) else None

'three' if isinstance(re_, np.ndarray) else None

one


'three'

In [65]:
def fit_transformer(request):
    DATA = bike.copy() #원본데이터 
    DF = pd.DataFrame()
    NUM = 5 #전치리된 결과를 보여줄 row 갯수 

    for user_request in request:
        get_request_dict = request[user_request] #전처리 테스트 요청 (전처리번호, 필드이름, 조건(있으면)) (딕셔너리 형태)
        get_pfunc = pfunc[str(get_request_dict['pfunc_pk'])] #db에서 가져온 전처리 정보 (딕셔너리 형태 )
        get_field = DATA[get_request_dict['field_name']].astype(float) #전처리를 수행할 필드
        transformer = get_transformer(get_pfunc) #전처리를 수행할 변환기 가져오기 
        try:
            get_condition = get_request_dict['condition'] #조건 (있으면)
            transformer = change_params(transformer, get_condition) #조건 (있으면 변환기 파라미터 수정해서 가져오기)
        except:
            pass
        print('{} 필드의 {} 전처리를 수행....'.format(get_request_dict['field_name'], get_pfunc['LIBRARY_FUNCTION_NAME']))
        try:
            transformer.fit(get_field)
            changed_field = transformer.transform(get_field)
        except:
            transformer.fit(get_field.values.reshape(-1,1))
            changed_field = transformer.transform(get_field.values.reshape(-1,1))
        if changed_field[:NUM].shape == (5,1):
            changed_field = list(map(lambda x:format(x, '.4f'), list(flatten(changed_field[:NUM]))))
        else:
            changed_field = list(map(lambda x:str(x), changed_field[:NUM]))

        to_df = pd.DataFrame(changed_field, columns=[get_request_dict['field_name']])
        DF = pd.concat([DF, to_df], axis=1)
    result = DF.to_dict(orient='records') 
    return result

get_result = fit_transformer(request=request)
get_result

atemp 필드의 StandardScaler 전처리를 수행....
humidity 필드의 MinMaxScaler 전처리를 수행....
temp 필드의 StandardScaler 전처리를 수행....
weather 필드의 OneHotEncoder 전처리를 수행....


[{'atemp': '-1.0927',
  'humidity': '0.8100',
  'temp': '-1.3337',
  'weather': '  (0, 0)\t1.0'},
 {'atemp': '-1.1824',
  'humidity': '0.8000',
  'temp': '-1.4389',
  'weather': '  (0, 0)\t1.0'},
 {'atemp': '-1.1824',
  'humidity': '0.8000',
  'temp': '-1.4389',
  'weather': '  (0, 0)\t1.0'},
 {'atemp': '-1.0927',
  'humidity': '0.7500',
  'temp': '-1.3337',
  'weather': '  (0, 0)\t1.0'},
 {'atemp': '-1.0927',
  'humidity': '0.7500',
  'temp': '-1.3337',
  'weather': '  (0, 0)\t1.0'}]

In [617]:
from collections import defaultdict 

DATA = bike.copy() #원본데이터 
DF = pd.DataFrame()
#result = defaultdict(list)
NUM = 5 #전치리된 결과를 보여줄 row 갯수 

for user_request in request:
    get_request_dict = request[user_request] #전처리 테스트 요청 (전처리번호, 필드이름, 조건(있으면)) (딕셔너리 형태)
    #{'pfunc_pk': 1, 'field_name': 'atemp', 'condition': ''}
    get_pfunc = pfunc[str(get_request_dict['pfunc_pk'])] #db에서 가져온 전처리 정보 (딕셔너리 형태 )
    #{'pk': 1, 'LIBRARY_NAME': 'sklearn', 'LIBRARY_OBJECT_NAME': 'preprocessing', 'LIBRARY_FUNCTION_NAME': 'StandardScaler'}
    get_field = DATA[get_request_dict['field_name']].astype(float) #전처리를 수행할 필드
    
    transformer = get_transformer(get_pfunc) #전처리를 수행할 변환기 가져오기 
    try:
        get_condition = get_request_dict['condition'] #조건 (있으면)
        transformer = change_params(transformer, get_condition) #조건 (있으면 변환기 파라미터 수정해서 가져오기)
    except:
        pass
        
    print('{} 필드의 {} 전처리를 수행....'.format(get_request_dict['field_name'], get_pfunc['LIBRARY_FUNCTION_NAME']))

    try:
        transformer.fit(get_field)
        changed_field = transformer.transform(get_field)
    except:
        transformer.fit(get_field.values.reshape(-1,1))
        changed_field = transformer.transform(get_field.values.reshape(-1,1))
        
    
    if changed_field[:NUM].shape == (5,1):
        changed_field = list(map(lambda x:format(x, '.4f'), list(flatten(changed_field[:NUM]))))
    else:
        changed_field = list(map(lambda x:str(x), changed_field[:NUM]))
        
    print(changed_field)
    to_df = pd.DataFrame(changed_field, columns=[get_request_dict['field_name']])
    DF = pd.concat([DF, to_df], axis=1)
    #for k, v in enumerate(changed_field):
    #    result[k].append(v)
print('\n ==============done==============\n')

    
print('response!')
result = DF.to_dict(orient='records') 
result
#result = dict(result)
#result

atemp 필드의 StandardScaler 전처리를 수행....
['-1.0927', '-1.1824', '-1.1824', '-1.0927', '-1.0927']
humidity 필드의 MinMaxScaler 전처리를 수행....
['0.6200', '0.6000', '0.6000', '0.5000', '0.5000']
temp 필드의 StandardScaler 전처리를 수행....
['-1.3337', '-1.4389', '-1.4389', '-1.3337', '-1.3337']
weather 필드의 OneHotEncoder 전처리를 수행....
['[1. 0. 0. 0.]', '[1. 0. 0. 0.]', '[1. 0. 0. 0.]', '[1. 0. 0. 0.]', '[1. 0. 0. 0.]']

 ==============done==============

response!


[{'atemp': '-1.0927',
  'humidity': '0.6200',
  'temp': '-1.3337',
  'weather': '[1. 0. 0. 0.]'},
 {'atemp': '-1.1824',
  'humidity': '0.6000',
  'temp': '-1.4389',
  'weather': '[1. 0. 0. 0.]'},
 {'atemp': '-1.1824',
  'humidity': '0.6000',
  'temp': '-1.4389',
  'weather': '[1. 0. 0. 0.]'},
 {'atemp': '-1.0927',
  'humidity': '0.5000',
  'temp': '-1.3337',
  'weather': '[1. 0. 0. 0.]'},
 {'atemp': '-1.0927',
  'humidity': '0.5000',
  'temp': '-1.3337',
  'weather': '[1. 0. 0. 0.]'}]

In [623]:
fresult =  dict(zip(range(len(result)), result))
pprint(fresult)

{0: {'atemp': '-1.0927',
     'humidity': '0.6200',
     'temp': '-1.3337',
     'weather': '[1. 0. 0. 0.]'},
 1: {'atemp': '-1.1824',
     'humidity': '0.6000',
     'temp': '-1.4389',
     'weather': '[1. 0. 0. 0.]'},
 2: {'atemp': '-1.1824',
     'humidity': '0.6000',
     'temp': '-1.4389',
     'weather': '[1. 0. 0. 0.]'},
 3: {'atemp': '-1.0927',
     'humidity': '0.5000',
     'temp': '-1.3337',
     'weather': '[1. 0. 0. 0.]'},
 4: {'atemp': '-1.0927',
     'humidity': '0.5000',
     'temp': '-1.3337',
     'weather': '[1. 0. 0. 0.]'}}


In [403]:
encoder = OneHotEncoder(categories='auto')
encoder

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [450]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(categories='auto', sparse=False)
encoder.fit(DATA['weather'].values.reshape(-1,1))
changed_field = encoder.transform(DATA['weather'].values.reshape(-1,1))
changed_field[:5]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [464]:
result = list(map(lambda x:str(x), changed_field[:5]))
result[0]

'[1. 0. 0. 0.]'

In [370]:
aa = list(map(lambda x:format(x, '.4f'), list(flatten(changed_field))))
aa

['0.8165', '-1.2247', '-1.2247', '0.8165', '0.8165']

In [348]:
for row in result:
    print(row)

{'atemp': 0.816496580927727, 'humidity': 1.0, 'temp': 0.8164965809277243}
{'atemp': -1.224744871391588, 'humidity': 0.6666666666666643, 'temp': -1.2247448713915907}
{'atemp': -1.224744871391588, 'humidity': 0.6666666666666643, 'temp': -1.2247448713915907}
{'atemp': 0.816496580927727, 'humidity': -1.0, 'temp': 0.8164965809277243}
{'atemp': 0.816496580927727, 'humidity': -1.0, 'temp': 0.8164965809277243}


In [242]:
get_field = DATA[['atemp', 'humidity']][:10]  #atemp, humidity, [['atemp', 'humidity']]
get_field

,atemp,humidity
0,14.395,81
1,13.635,80
2,13.635,80
3,14.395,75
4,14.395,75
5,12.880,75
6,13.635,80
7,12.880,86
8,14.395,75
9,17.425,76


In [235]:
get_field=get_field.astype(float)
try:
    print('here')
    transformer.fit(get_field)
    print('here')
    changed_f1 = transformer.fit_transform(get_field)
except:
    print('here2')
    transformer.fit(get_field.values.reshape(-1,1))
    print('here2')
    changed_f1 = transformer.fit_transform(get_field.values.reshape(-1,1))
changed_f1

here
here


array([[-0.33333333,  0.09090909],
       [-0.66776678, -0.09090909],
       [-0.66776678, -0.09090909],
       [-0.33333333, -1.        ],
       [-0.33333333, -1.        ],
       [-1.        , -1.        ],
       [-0.66776678, -0.09090909],
       [-1.        ,  1.        ],
       [-0.33333333, -1.        ],
       [ 1.        , -0.81818182]])

In [20]:
def get_transformer(params):
    module_ = __import__(params['LIBRARY_NAME'])
    class_ = getattr(module_, params['LIBRARY_OBJECT_NAME'])
    transformer_ = getattr(class_, params['LIBRARY_FUNCTION_NAME'])
    transformer  = transformer_()
    return transformer   

def change_params(model, param):
    try:
        for k, v in param.items():
            setattr(model, k, v)
        return model

    except:
        return model

In [83]:
from sklearn.linear_model import Ridge 
from sklearn.ensemble import RandomForestClassifier
clf = Ridge()
clf_ = RandomForestClassifier()
user_param = '{"normalize":"True"}'
print(type(user_param))
user_param = json.loads(user_param)
print(type(user_param))
clf2 = change_params(clf, user_param)
clf2

<class 'str'>
<class 'dict'>


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize='True', random_state=None, solver='auto', tol=0.001)

In [84]:
clf_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
#scale_fit = mm_scaler.fit(DATA[COLUMN].values.reshape(-1,1))
#x_scaled = scale_fit.transform(DATA[COLUMN].values.reshape(-1,1))
x_scaled = mm_scaler.fit_transform(DATA[COLUMN].values.reshape(-1,1))
print(scale_fit.n_samples_seen_)
x_scaled[:5]

10886


array([[0.30506768],
       [0.28806354],
       [0.28806354],
       [0.30506768],
       [0.30506768]])

In [32]:
bike.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.39,81,0.00,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.63,80,0.00,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.63,80,0.00,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.39,75,0.00,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.39,75,0.00,0,1,1


In [34]:
bike.tail()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.70,50,26.00,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.43,57,15.00,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.91,61,15.00,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.43,61,6.00,12,117,129
10885,2012-12-19 23:00:00,4,0,1,1,13.12,16.66,66,9.00,4,84,88


In [33]:
DATA[COLUMN] = x_scaled
DATA.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,0.31,81,0.00,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,0.29,80,0.00,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,0.29,80,0.00,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,0.31,75,0.00,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,0.31,75,0.00,0,1,1


In [28]:
pd.options.display.float_format = '{:.2f}'.format

In [9]:
x_scaled

array([[0.30506768],
       [0.28806354],
       [0.28806354],
       ...,
       [0.33896409],
       [0.3728605 ],
       [0.35585636]])

In [10]:
scaled_02 = scale_fit.transform(unseen_data)
scaled_02

array([[0.30506768],
       [0.28806354],
       [0.28806354],
       [0.30506768],
       [0.30506768]])

In [14]:
import joblib

def save_estimator(estimator, estimator_name):
    file_path = os.path.join(current_dir, estimator_name)
    joblib.dump(estimator, file_path)
    return file_path, estimator_name

def load_estimator(model_name):
    file_path = os.path.join(current_dir, model_name)
    clf = joblib.load(file_path)
    return clf

In [15]:
path, fname = save_estimator(mm_scaler, 'mm_scaler.pickle')
path

'C:\\Users\\Daumsoft\\Desktop\\mm_scaler.pickle'

In [16]:
load_scaler = load_estimator('mm_scaler.pickle')
load_scaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [17]:
load_scaler.n_samples_seen_

10886

In [18]:
unseen_data

array([[14.395],
       [13.635],
       [13.635],
       [14.395],
       [14.395]])

In [19]:
unseen_scaled = load_scaler.transform(unseen_data)
unseen_scaled

array([[0.30506768],
       [0.28806354],
       [0.28806354],
       [0.30506768],
       [0.30506768]])

In [18]:
load_scaler.n_samples_seen_

10886